In [4]:
import numpy as np
import pandas as pd
import ast
from gensim.models import Word2Vec
from tensorflow
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F

2022-11-21 15:58:29.636766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 15:58:29.791531: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-21 15:58:29.824224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-21 15:58:29.824233: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [6]:
mal_fam_cont = {'mirai': 0, 'unknown': 0, 'penguin': 0, 'fakebank': 0, 'fakeinst': 0, 'mobidash': 0, 'berbew': 0, 'wroba': 0}

In [12]:
dataset = pd.read_csv(r'/mnt/bigDisk/weiren/Labels_TimeSyscallSeqs.csv')

In [13]:
new_dataset = []
# drop_fam = ['metasploit','zbot','skeeyah','hiddad']
drop_index = []
dataset = dataset.drop(['CLASS', 'BEH'], axis=1)
for i in range(len(dataset)):
    if dataset['FAM'][i] in mal_fam_cont and mal_fam_cont[dataset['FAM'][i]] < 10:
        mal_fam_cont[dataset['FAM'][i]] += 1
    else:
        drop_index.append(i)
        
dataset = dataset.drop(drop_index)

In [14]:
dataset = dataset.reset_index(drop = True)

In [15]:
dataset

,FILENAME,FAM,SEQUENCE
0,ca970cc75ccdf7bd1959b970de1f81d682ea3af871aff6...,mirai,"[('1659223215.715424', 'execve'), ('1659223215..."
1,699ffec8df22106524defd950bff835735051514a9b059...,mirai,"[('1662079445.156731', 'execve'), ('1662079445..."
2,94e883f37be5e6a2791b081e78c86804b427e90e55a172...,mirai,"[('946684902.916742', 'execve'), ('946684903.7..."
3,6b8490cc0ea6b234de8b6529b9277ee0ccba22026b4689...,unknown,"[('1970.5854504280915968', 'wait4'), ('1970.62..."
4,93a9669eea9b49258a1c4ca20b261561b6bd91b43643ad...,mirai,"[('1970.-1293549660274688', 'read'), ('1970.-1..."
...,...,...,...
75,94a85f288f8e88b821b61bcb1caa08c1f38230c9da0a11...,penguin,"[('1970.000000', 'execve'), ('2022.064030', 'c..."
76,814e3aa1aec837c9d9e4e2ae03f66cd821fb648237cf69...,penguin,"[('1970.000000', 'execve'), ('2022.052163', 'b..."
77,334a894d9e7e7d8b3869212e5a77b2a897b6a7171b5f4e...,penguin,"[('1970.000000', 'execve'), ('2022.051174', 'n..."
78,b468d1f8084fbefa850888d87e42ad5f30079547b66823...,penguin,"[('1970.000000', 'execve'), ('2022.068917', 'b..."


In [16]:
mal_fam_label = {'mirai': 0, 'unknown': 1, 'penguin': 2, 'fakebank': 3, 'fakeinst': 4, 'mobidash': 5, 'berbew': 6, 'wroba': 7}
label = []
for fam in dataset.FAM:
    label.append(mal_fam_label[fam])

In [17]:
def ExtractSyscallSeq(SeqStr):
    SeqStr = ast.literal_eval(SeqStr)
    return [syscall for time, syscall in SeqStr]

In [18]:
Seq = list()
Seq = [ ExtractSyscallSeq(seq) for seq in dataset.SEQUENCE ]

In [19]:
max_len = 0
for i in range(80):
    max_len = max(max_len, len(Seq[i]))

In [20]:
#ngram word2vec
from gensim.models import Phrases

bigram_transformer = Phrases(Seq)

model = Word2Vec(bigram_transformer[Seq], min_count = 1)
word_dict = dict(zip(model.wv.index_to_key, model.wv.vectors))

In [30]:
X = []
for i in Seq:
    tmp = np.zeros(100)
    for j in i:
        tmp += word_dict[j]
    X.append(tmp)
X = np.array(X) / np.max(np.array(X), axis=0)
# X = np.expand_dims(X, axis=1)
# X.shape

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, label, test_size=0.2, shuffle=True)

In [34]:
print('len of training set : ', len(X_train))
print('len of testing set  : ', len(X_test))

len of training set :  64
len of testing set  :  16


In [28]:
# class CNN_model(torch.nn.Module):
#     def __init__(self, input_size, hidden_dim, output_size):
#         super(CNN_model, self).__init__()
#         self.conv1 = torch.nn.Conv1d(1, 1, kernel_size=5, stride=1, padding="same")
#         self.conv2 = torch.nn.Conv1d(1, 1, kernel_size=3, stride=1, padding="same")

#         self.mx_pool1 = torch.nn.MaxPool1d(kernel_size=4, stride=4)
        
#         self.fc1 = torch.nn.Linear(25, hidden_dim)
#         self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
#         # self.fc3 = torch.nn.Linear(hidden_dim, hidden_dim)
#         self.out = torch.nn.Linear(hidden_dim, output_size)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.conv2(x)
#         x = self.mx_pool1(x)

#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         # x = self.fc3(x)
#         # x = F.relu(x)
#         x = self.out(x)
#         x = F.softmax(x)
#         # print(x)

#         return x[0]

In [29]:
class SIMPLE_model(torch.nn.Module):
    def __init__(self, input_size , hidden_dim, output_size):
        super(SIMPLE_model,self).__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_dim // 2, num_layers = 1, bidirectional=True)
        self.Conv1 = torch.nn.Conv1d(1, hidden_dim, kernel_size = 3, stride=1, padding="same")
        self.pool1 = torch.nn.MaxPool1d(kernel_size = 2, stride=2)
        self.Conv2 = torch.nn.Conv1d(64, 128, kernel_size = 3, stride=1,padding="same")
        self.fc1 = torch.nn.Linear(4096,512)
        self.fc2 = torch.nn.Linear(512,64)
        self.fc5 = torch.nn.Linear(64,output_size)
        
    def forward(self,din):
        din = din.unsqueeze(1)
        dout,_ = self.lstm(din)
        dout = F.relu(self.Conv1(dout))
        dout = self.pool1(dout)
        dout = F.relu(self.Conv2(dout))
        dout = torch.flatten(dout)
        dout = F.relu(self.fc1(dout))
        dout = F.relu(self.fc2(dout))
        dout = F.softmax(self.fc5(dout))
        return dout

In [30]:
Y_train_vec = []
for i in range(len(Y_train)):
    y_temp = [0]*8 
    y_temp[Y_train[i]] = 1
    Y_train_vec.append(y_temp)

In [31]:
Y_test_vec = []
for i in range(len(Y_test)):
    y_temp = [0]*8 
    y_temp[Y_test[i]] = 1
    Y_test_vec.append(y_temp)

In [32]:
X

array([[ 1.00000000e+00,  5.56423760e-01,  6.08926700e-01, ...,
        -6.24673480e+00, -1.04063707e+01,  6.27349198e-01],
       [-1.06470419e+02,  3.92147183e-01,  1.00000000e+00, ...,
        -6.27156374e+01, -7.20526821e+00,  6.53489751e-01],
       [-1.86003191e+00, -2.43760189e-02,  2.95063414e-02, ...,
        -1.56393462e+00,  1.00000000e+00,  1.61834914e-01],
       ...,
       [-4.40248877e+00,  1.93889308e-01,  1.32902011e-03, ...,
         8.79840593e-01, -5.51048709e-01, -2.95078984e-02],
       [-4.34413893e+00,  1.92040289e-01,  1.59262855e-03, ...,
         8.75569059e-01, -5.48593543e-01, -2.88008129e-02],
       [-4.51685266e+00,  2.00533404e-01,  1.93706134e-03, ...,
         9.10976622e-01, -5.85875480e-01, -3.10542559e-02]])

In [34]:
lbl, cnt = np.unique(Y_train, return_counts=True)
print(dict(zip(lbl, cnt)))

lbl_test, cnt_test = np.unique(Y_test, return_counts=True)
print(dict(zip(lbl_test, cnt_test)))

{0: 8, 1: 7, 2: 8, 3: 8, 4: 8, 5: 7, 6: 9, 7: 9}
{0: 2, 1: 3, 2: 2, 3: 2, 4: 2, 5: 3, 6: 1, 7: 1}


In [35]:
train_features_init = np.array(X_train)
train_label_init = np.array(Y_train_vec)
train_features_init = torch.from_numpy(train_features_init)
train_label_init = np.array(train_label_init).astype('float32') # 轉換成 nparray
train_label_init = torch.from_numpy(train_label_init).to(torch.float)


strace_train = torch.utils.data.TensorDataset(train_features_init,train_label_init)

#Testdata
test_features_init = np.array(X_test)
test_label_init = np.array(Y_test_vec)
test_features_init = torch.from_numpy(test_features_init)
test_label_init = np.array(test_label_init).astype('float32') # 轉換成 nparray
test_label_init = torch.from_numpy(test_label_init).to(torch.float)

strace_test = torch.utils.data.TensorDataset(test_features_init, test_label_init)


In [36]:
strace_train.tensors[0].shape

torch.Size([64, 100])

In [37]:
n_epochs = 10000     # epoch 的数目
batch_size = 1  # 决定每次读取多少图10
train_loader = torch.utils.data.DataLoader(strace_train, batch_size = batch_size, num_workers = 0)
test_loader = torch.utils.data.DataLoader(strace_test, batch_size = batch_size, num_workers = 0)
# print(strace_train.shape)

In [38]:
def test():
    correct = 0
    with torch.no_grad():  # test集中不需要反向传播
        for data in test_loader:
            images, labels = data
            outputs = model(images.float())
            lbl = torch.argmax(labels, dim=1)[0]
            out = torch.argmax(outputs)
            correct += (out==lbl)
            # print('correct: ', correct)
    print('Accuracy of the network on the test data: %d %%' % (
        100 * correct / len(test_loader.dataset)))
    return 100.0 * correct / len(test_loader.dataset)

In [39]:
def train_acc():
    correct = 0
    with torch.no_grad():  # test集中不需要反向传播
        for data in train_loader:
            images, labels = data
            outputs = model(images.float())
            lbl = torch.argmax(labels, dim=1)[0]
            out = torch.argmax(outputs)
            correct += (out==lbl)
            # print('correct: ', correct)
    print('Accuracy of the network on the train data: %d %%' % (
        100 * correct / len(train_loader.dataset)))
    return 100.0 * correct / len(train_loader.dataset)

In [40]:
def train():
    lossfunc = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.0001, weight_decay=0)
    for epoch in range(n_epochs):
        train_loss = 0.0
        for data,target in train_loader:
            optimizer.zero_grad()
            output = model(data.float())
            loss = lossfunc(output,target[0])
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
        train_acc()
        test()

In [45]:
# model = CNN_model(input_size=100, hidden_dim=256, output_size=8)
model = SIMPLE_model(input_size=100, hidden_dim=64, output_size=8)

In [46]:
(next(iter(train_loader.dataset))[1]).shape

torch.Size([8])

In [47]:
train()

/tmp/ipykernel_184407/3811027734.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  dout = F.softmax(self.fc5(dout))  # 输出层使用 softmax 激活函数


Epoch:  1  	Training Loss: 2.081234
Accuracy of the network on the train data: 18 %
Accuracy of the network on the test data: 25 %
Epoch:  2  	Training Loss: 2.077631
Accuracy of the network on the train data: 23 %
Accuracy of the network on the test data: 31 %
Epoch:  3  	Training Loss: 2.070511
Accuracy of the network on the train data: 28 %
Accuracy of the network on the test data: 25 %
Epoch:  4  	Training Loss: 2.051791
Accuracy of the network on the train data: 26 %
Accuracy of the network on the test data: 31 %
Epoch:  5  	Training Loss: 2.014641
Accuracy of the network on the train data: 37 %
Accuracy of the network on the test data: 31 %
Epoch:  6  	Training Loss: 1.963036
Accuracy of the network on the train data: 39 %
Accuracy of the network on the test data: 25 %
Epoch:  7  	Training Loss: 1.906331
Accuracy of the network on the train data: 39 %
Accuracy of the network on the test data: 18 %
Epoch:  8  	Training Loss: 1.882278
Accuracy of the network on the train data: 39 %